In [20]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)
from torchvision.models import AlexNet_Weights

In [18]:
def validate_model(model_name, model_path, val_path):
    # Hardware configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Validation data configuration
    val_dir = val_path
    if not os.path.exists(val_dir):
        raise FileNotFoundError(f"Validation directory not found: {val_dir}")

    # Validation transforms
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Create dataset and loader
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Model initialization
    match model_name:
        case 'alexnet':
            model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
    match model_name:
        case 'densenet':
            model = models.densenet(pretrained=True)
    match model_name:
        case 'googlenet':
            model = models.googlenet(pretrained=True)
    match model_name:
        case 'resnet50':
            model = models.resnet50(pretrained=True)
    match model_name:
        case 'mobilenet':
            model = models.mobilenet(pretrained=True)
    match model_name:
        case 'improved':
            model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

    
    # Freeze feature parameters
    # for param in model.features.parameters():
    #     param.requires_grad = False
    
    # Modify classifier
    num_classes = len(val_dataset.classes)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    
    # Load trained weights
    model_path = model_path
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    # Initialize metrics
    all_preds = []
    all_labels = []
    all_probs = []
    running_corrects = 0

    # Validation loop
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Get class probabilities
            
            _, preds = torch.max(outputs, 1)
            
            running_corrects += torch.sum(preds == labels.data)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Calculate metrics
    val_acc = running_corrects.double() / len(val_dataset)
    sk_acc = accuracy_score(all_labels, all_preds)
    val_precision = precision_score(all_labels, all_preds, average='macro')
    val_recall = recall_score(all_labels, all_preds, average='macro')
    val_f1 = f1_score(all_labels, all_preds, average='macro')
    val_roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    # Print comprehensive results
    print(f"\nValidation Metrics:")
    print(f"Manual Accuracy: {val_acc:.4f}")
    print(f"Sklearn Accuracy: {sk_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall: {val_recall:.4f}")
    print(f"F1-Score: {val_f1:.4f}")
    print(f"ROC-AUC: {val_roc_auc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

In [21]:
# alexnet
if __name__ == "__main__":
    validate_model('alexnet', '../model/9444/alexnet_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')

Using device: cuda

Validation Metrics:
Manual Accuracy: 0.8781
Sklearn Accuracy: 0.8781
Precision: 0.8544
Recall: 0.7197
F1-Score: 0.7539
ROC-AUC: 0.9751

Confusion Matrix:
[[112   1   6   0   0]
 [  5   8   7   0   0]
 [  2   0  85   1   3]
 [  0   0   0   4   6]
 [  1   0   2   0  36]]


In [15]:
# densenet
if __name__ == "__main__":
    validate_model('densenet', '../model/9444/densenet161_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')

Using device: cuda


TypeError: 'module' object is not callable

In [16]:
# googlenet
if __name__ == "__main__":
    validate_model('googlenet', '../model/9444/googlenet_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')

Using device: cuda
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\99320/.cache\torch\hub\checkpoints\googlenet-1378be20.pth


D:\Environment\Anaconda\envs\UNSW\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100.0%


AttributeError: 'GoogLeNet' object has no attribute 'classifier'

In [12]:
# resnet50
if __name__ == "__main__":
    validate_model('resnet50', '../model/9444/resnet50_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')

Using device: cuda


D:\Environment\Anaconda\envs\UNSW\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Environment\Anaconda\envs\UNSW\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'ResNet' object has no attribute 'classifier'

In [17]:
# mobilenet
if __name__ == "__main__":
    validate_model('mobilenet', '../model/9444/mobilenet_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')

Using device: cuda


TypeError: 'module' object is not callable

In [ ]:
# improved
if __name__ == "__main__":
    validate_model('../model/9444/improved_EBHI_mag_200.pth', '../EBH-HE-IDS/ColHis-IDS_split/test/200')